# Realizando Operações Estilo Banco de Dados em Dataframes

## Sobre os dados
Neste notebook, usaremos dados meteorológicos diários retirados da [API do National Centers for Environmental Information (NCEI)](https://www.ncdc.noaa.gov/cdo-web/webservices/v2). O notebook [`0-weather_data_collection.ipynb`](./0-weather_data_collection.ipynb) contém o processo seguido para coletar os dados. Consulte a [documentação](https://www1.ncdc.noaa.gov/pub/data/cdo/documentation/GHCND_documentation.pdf) do conjunto de dados para obter informações sobre os campos.

*Nota: O NCEI faz parte da National Oceanic and Atmospheric Administration (NOAA) e, como você pode ver pela URL da API, este recurso foi criado quando o NCEI era chamado de NCDC. Caso a URL deste recurso mude no futuro, você pode procurar por "API meteorológica NCEI" para encontrar a nova URL.*

## Configuração

In [1]:
import pandas as pd

weather = pd.read_csv('data/nyc_weather_2018.csv')
weather.head()

,date,datatype,station,attributes,value
0,2018-01-01T00:00:00,PRCP,GHCND:US1CTFR0039,",,N,",0.0
1,2018-01-01T00:00:00,PRCP,GHCND:US1NJBG0015,",,N,",0.0
2,2018-01-01T00:00:00,SNOW,GHCND:US1NJBG0015,",,N,",0.0
3,2018-01-01T00:00:00,PRCP,GHCND:US1NJBG0017,",,N,",0.0
4,2018-01-01T00:00:00,SNOW,GHCND:US1NJBG0017,",,N,",0.0


## Consultando DataFrames
O método `query()` é uma maneira mais fácil de filtrar com base em alguns critérios. Por exemplo, podemos usá-lo para encontrar todas as entradas onde a neve foi registrada em uma estação com `US1NY` no seu ID de estação:

In [2]:
snow_data = weather.query('datatype == "SNOW" and value > 0 and station.str.contains("US1NY")')
snow_data.head()

,date,datatype,station,attributes,value
114,2018-01-01T00:00:00,SNOW,GHCND:US1NYWC0019,",,N,",25.0
789,2018-01-04T00:00:00,SNOW,GHCND:US1NYNS0007,",,N,",41.0
794,2018-01-04T00:00:00,SNOW,GHCND:US1NYNS0018,",,N,",10.0
798,2018-01-04T00:00:00,SNOW,GHCND:US1NYNS0024,",,N,",89.0
800,2018-01-04T00:00:00,SNOW,GHCND:US1NYNS0030,",,N,",102.0


Isso é equivalente a consultar o banco de dados SQLite `weather.db` com 

```sql
SELECT * 
FROM weather 
WHERE datatype == "SNOW" AND value > 0 AND station LIKE "%US1NY%"
```

In [4]:
import sqlite3

with sqlite3.connect('data/weather.db') as connection:
    snow_data_from_db = pd.read_sql(
        'SELECT * FROM weather WHERE datatype == "SNOW" AND value > 0 AND station LIKE "%US1NY%"', 
        connection
    )

snow_data.reset_index().drop(columns='index').equals(snow_data_from_db)

True

## Mesclando DataFrames
Temos dados de várias estações diferentes todos os dias; no entanto, não sabemos quais são as estações—apenas seus IDs. Podemos juntar os dados no arquivo `weather_stations.csv`, que contém informações do endpoint `stations` da API do NCEI. Consulte o notebook [`0-weather_data_collection.ipynb`](./0-weather_data_collection.ipynb) para ver como isso foi coletado. Ele se parece com isto:

In [8]:
station_info = pd.read_csv('data/weather_stations.csv')
station_info.head()

,id,name,latitude,longitude,elevation
0,GHCND:US1CTFR0022,"STAMFORD 2.6 SSW, CT US",41.064100,-73.577000,36.6
1,GHCND:US1CTFR0039,"STAMFORD 4.2 S, CT US",41.037788,-73.568176,6.4
2,GHCND:US1NJBG0001,"BERGENFIELD 0.3 SW, NJ US",40.921298,-74.001983,20.1
3,GHCND:US1NJBG0002,"SADDLE BROOK TWP 0.6 E, NJ US",40.902694,-74.083358,16.8
4,GHCND:US1NJBG0003,"TENAFLY 1.3 W, NJ US",40.914670,-73.977500,21.6


Para relembrar, os dados meteorológicos se parecem com isto:

In [9]:
weather.head()

,date,datatype,station,attributes,value
0,2018-01-01T00:00:00,PRCP,GHCND:US1CTFR0039,",,N,",0.0
1,2018-01-01T00:00:00,PRCP,GHCND:US1NJBG0015,",,N,",0.0
2,2018-01-01T00:00:00,SNOW,GHCND:US1NJBG0015,",,N,",0.0
3,2018-01-01T00:00:00,PRCP,GHCND:US1NJBG0017,",,N,",0.0
4,2018-01-01T00:00:00,SNOW,GHCND:US1NJBG0017,",,N,",0.0


Podemos juntar nossos dados correspondendo a coluna `station_info.id` com a coluna `weather.station`. Antes de fazer isso, vamos ver quantos valores únicos temos:

In [10]:
station_info.id.describe()

count                   279
unique                  279
top       GHCND:US1CTFR0022
freq                      1
Name: id, dtype: object

Enquanto `station_info` tem uma linha por estação, o dataframe `weather` tem muitas entradas por estação. Observe que também possui menos valores únicos:

In [11]:
weather.station.describe()

count                 78780
unique                  110
top       GHCND:USW00094789
freq                   4270
Name: station, dtype: object

Ao trabalhar com joins, é importante ficar de olho na contagem de linhas. Alguns tipos de join podem levar à perda de dados. Lembre-se de que podemos obter essa contagem usando `shape`:

In [12]:
station_info.shape[0], weather.shape[0]

(279, 78780)

Como faremos isso com frequência, faz mais sentido escrever uma função:

In [13]:
def get_row_count(*dfs):
    return [df.shape[0] for df in dfs]
get_row_count(station_info, weather)

[279, 78780]

Por padrão, o `merge()` realiza uma junção interna (`inner`). Simplesmente especificamos as colunas a serem usadas para a junção. O dataframe da esquerda é aquele no qual chamamos o `merge()`, e o da direita é passado como argumento:

In [15]:
inner_join = weather.merge(station_info, left_on='station', right_on='id')
inner_join.sample(5, random_state=0)

,date,datatype,station,attributes,value,id,name,latitude,longitude,elevation
10739,2018-02-17T00:00:00,PRCP,GHCND:USC00066655,",,7,0700",4.1,GHCND:USC00066655,"PUTNAM LAKE, CT US",41.082500,-73.638600,91.4
45188,2018-07-27T00:00:00,SNOW,GHCND:US1NJES0019,",,N,",0.0,GHCND:US1NJES0019,"WEST CALDWELL TWP 1.3 NE, NJ US",40.861500,-74.277500,81.4
59823,2018-10-05T00:00:00,PRCP,GHCND:US1NJES0024,",,N,",0.0,GHCND:US1NJES0024,"CEDAR GROVE TWP 0.4 W, NJ US",40.855695,-74.235564,108.5
10852,2018-02-17T00:00:00,TMIN,GHCND:USW00094789,",,W,2400",-2.1,GHCND:USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.639150,-73.764010,3.4
46755,2018-08-03T00:00:00,AWND,GHCND:USW00094745,",,W,",1.8,GHCND:USW00094745,"WESTCHESTER CO AIRPORT, NY US",41.062360,-73.704630,111.9


Podemos remover a duplicação de informações nas colunas `station` e `id` renomeando uma delas antes da mesclagem e então simplesmente usando `on`:

In [16]:
weather.merge(station_info.rename({'id':'station'}, axis=1), on='station').sample(5, random_state=0)

,date,datatype,station,attributes,value,name,latitude,longitude,elevation
10739,2018-02-17T00:00:00,PRCP,GHCND:USC00066655,",,7,0700",4.1,"PUTNAM LAKE, CT US",41.082500,-73.638600,91.4
45188,2018-07-27T00:00:00,SNOW,GHCND:US1NJES0019,",,N,",0.0,"WEST CALDWELL TWP 1.3 NE, NJ US",40.861500,-74.277500,81.4
59823,2018-10-05T00:00:00,PRCP,GHCND:US1NJES0024,",,N,",0.0,"CEDAR GROVE TWP 0.4 W, NJ US",40.855695,-74.235564,108.5
10852,2018-02-17T00:00:00,TMIN,GHCND:USW00094789,",,W,2400",-2.1,"JFK INTERNATIONAL AIRPORT, NY US",40.639150,-73.764010,3.4
46755,2018-08-03T00:00:00,AWND,GHCND:USW00094745,",,W,",1.8,"WESTCHESTER CO AIRPORT, NY US",41.062360,-73.704630,111.9


Estamos perdendo estações que não têm observações meteorológicas associadas a elas. Se não quisermos perder essas linhas, podemos realizar um left ou right join em vez do inner join:

In [17]:
left_join = station_info.merge(weather, left_on='id', right_on='station', how='left')
right_join = weather.merge(station_info, left_on='station', right_on='id', how='right')

right_join[right_join.datatype.isna()].head()

,date,datatype,station,attributes,value,id,name,latitude,longitude,elevation
0,NaN,NaN,NaN,NaN,NaN,GHCND:US1CTFR0022,"STAMFORD 2.6 SSW, CT US",41.064100,-73.577000,36.6
344,NaN,NaN,NaN,NaN,NaN,GHCND:US1NJBG0001,"BERGENFIELD 0.3 SW, NJ US",40.921298,-74.001983,20.1
345,NaN,NaN,NaN,NaN,NaN,GHCND:US1NJBG0002,"SADDLE BROOK TWP 0.6 E, NJ US",40.902694,-74.083358,16.8
718,NaN,NaN,NaN,NaN,NaN,GHCND:US1NJBG0005,"WESTWOOD 0.8 ESE, NJ US",40.983041,-74.015858,15.8
719,NaN,NaN,NaN,NaN,NaN,GHCND:US1NJBG0006,"RAMSEY 0.6 E, NJ US",41.058611,-74.134068,112.2


O left e o right join como fizemos acima são equivalentes porque o lado para o qual mantivemos as linhas sem correspondência foi o mesmo em ambos os casos:

In [18]:
left_join.sort_index(axis=1).sort_values(['date', 'station'], ignore_index=True).equals(
    right_join.sort_index(axis=1).sort_values(['date', 'station'], ignore_index=True)
)

True

Note que temos linhas adicionais nos left e right joins porque mantivemos todas as estações que não tinham observações meteorológicas:

In [19]:
get_row_count(inner_join, left_join, right_join)

[78780, 78949, 78949]

Se consultarmos as informações da estação para estações que têm `US1NY` em seu ID e realizarmos um outer join, podemos ver onde ocorrem as inconsistências:

In [20]:
outer_join = weather.merge(
    station_info[station_info.id.str.contains('US1NY')], 
    left_on='station', right_on='id', how='outer', indicator=True
)

pd.concat([
    outer_join.query(f'_merge == "{kind}"').sample(2, random_state=0) 
    for kind in outer_join._merge.unique()
]).sort_index()

,date,datatype,station,attributes,value,id,name,latitude,longitude,elevation,_merge
28719,2018-03-05T00:00:00,PRCP,GHCND:US1NYNS0037,",,N,",0.3,GHCND:US1NYNS0037,"WANTAGH 1.1 NNE, NY US",40.683311,-73.503837,10.4,both
30828,2018-12-19T00:00:00,PRCP,GHCND:US1NYNS0046,",,N,",0.0,GHCND:US1NYNS0046,"MASSAPEQUA PARK 1.2 N, NY US",40.698077,-73.449893,10.7,both
32004,NaN,NaN,NaN,NaN,NaN,GHCND:US1NYQN0033,"HOWARD BEACH 0.4 NNW, NY US",40.662099,-73.841345,2.1,right_only
34194,NaN,NaN,NaN,NaN,NaN,GHCND:US1NYWC0009,"NEW ROCHELLE 1.3 S, NY US",40.904000,-73.777000,21.9,right_only
62899,2018-07-21T00:00:00,TMAX,GHCND:USW00054787,",,W,",24.4,NaN,NaN,NaN,NaN,NaN,left_only
73018,2018-07-19T00:00:00,TMAX,GHCND:USW00094745,",,W,2400",27.2,NaN,NaN,NaN,NaN,NaN,left_only


Esses joins são equivalentes aos seus equivalentes em SQL. Abaixo está o exemplo do inner join. Note que para usar `equals()`, você precisará manipular os dataframes para alinhá-los:

In [21]:
import sqlite3

with sqlite3.connect('data/weather.db') as connection:
    inner_join_from_db = pd.read_sql(
        'SELECT * FROM weather JOIN stations ON weather.station == stations.id', 
        connection
    )

inner_join_from_db.shape == inner_join.shape

True

Revisitando os dados sujos do notebook [`5-handling_data_issues.ipynb`](../ch_03/5-handling_data_issues.ipynb) do capítulo 3.

Significados dos dados:
- `PRCP`: precipitação em milímetros
- `SNOW`: queda de neve em milímetros
- `SNWD`: profundidade de neve em milímetros
- `TMAX`: temperatura máxima diária em Celsius
- `TMIN`: temperatura mínima diária em Celsius
- `TOBS`: temperatura no momento da observação em Celsius
- `WESF`: equivalente de água da neve em milímetros

Leia os dados, removendo duplicatas e a coluna não informativa `SNWD`:

In [22]:
dirty_data = pd.read_csv(
    'data/dirty_data.csv', index_col='date'
).drop_duplicates().drop(columns='SNWD')
dirty_data.head()

,station,PRCP,SNOW,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01T00:00:00,?,0.0,0.0,5505.0,-40.0,NaN,NaN,NaN
2018-01-02T00:00:00,GHCND:USC00280907,0.0,0.0,-8.3,-16.1,-12.2,NaN,False
2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-4.4,-13.9,-13.3,NaN,False
2018-01-04T00:00:00,?,20.6,229.0,5505.0,-40.0,NaN,19.3,True
2018-01-05T00:00:00,?,0.3,NaN,5505.0,-40.0,NaN,NaN,NaN


Precisamos criar dois dataframes para a junção. Também vamos remover algumas colunas desnecessárias para facilitar a visualização:

In [23]:
valid_station = dirty_data.query('station != "?"').drop(columns=['WESF', 'station'])
station_with_wesf = dirty_data.query('station == "?"').drop(columns=['station', 'TOBS', 'TMIN', 'TMAX'])

In [25]:
valid_station.head()

,PRCP,SNOW,TMAX,TMIN,TOBS,inclement_weather
date,,,,,,
2018-01-02T00:00:00,0.0,0.0,-8.3,-16.1,-12.2,False
2018-01-03T00:00:00,0.0,0.0,-4.4,-13.9,-13.3,False
2018-01-05T00:00:00,14.2,127.0,-4.4,-13.9,-13.9,True
2018-01-06T00:00:00,0.0,0.0,-10.0,-15.6,-15.0,False
2018-01-07T00:00:00,0.0,0.0,-11.7,-17.2,-16.1,False


In [26]:
station_with_wesf.head()

,PRCP,SNOW,WESF,inclement_weather
date,,,,
2018-01-01T00:00:00,0.0,0.0,NaN,NaN
2018-01-04T00:00:00,20.6,229.0,19.3,True
2018-01-05T00:00:00,0.3,NaN,NaN,NaN
2018-01-12T00:00:00,0.5,NaN,NaN,NaN
2018-01-13T00:00:00,17.5,NaN,NaN,NaN


Nossa coluna para a junção é o índice em ambos os dataframes, então devemos especificar `left_index` e `right_index`:

In [27]:
valid_station.merge(
    station_with_wesf, how='left', left_index=True, right_index=True
).query('WESF > 0').head()

,PRCP_x,SNOW_x,TMAX,TMIN,TOBS,inclement_weather_x,PRCP_y,SNOW_y,WESF,inclement_weather_y
date,,,,,,,,,,
2018-01-30T00:00:00,0.0,0.0,6.7,-1.7,-0.6,False,1.5,13.0,1.8,True
2018-03-08T00:00:00,48.8,NaN,1.1,-0.6,1.1,False,28.4,NaN,28.7,NaN
2018-03-13T00:00:00,4.1,51.0,5.6,-3.9,0.0,True,3.0,13.0,3.0,True
2018-03-21T00:00:00,0.0,0.0,2.8,-2.8,0.6,False,6.6,114.0,8.6,True
2018-04-02T00:00:00,9.1,127.0,12.8,-1.1,-1.1,True,14.0,152.0,15.2,True


As colunas que existiam em ambos os dataframes, mas não fizeram parte da junção, tiveram sufixos adicionados aos seus nomes: `_x` para colunas do dataframe da esquerda e `_y` para colunas do dataframe da direita. Podemos personalizar isso com o argumento `suffixes`:

In [28]:
valid_station.merge(
    station_with_wesf, how='left', left_index=True, right_index=True, suffixes=('', '_?')
).query('WESF > 0').head()

,PRCP,SNOW,TMAX,TMIN,TOBS,inclement_weather,PRCP_?,SNOW_?,WESF,inclement_weather_?
date,,,,,,,,,,
2018-01-30T00:00:00,0.0,0.0,6.7,-1.7,-0.6,False,1.5,13.0,1.8,True
2018-03-08T00:00:00,48.8,NaN,1.1,-0.6,1.1,False,28.4,NaN,28.7,NaN
2018-03-13T00:00:00,4.1,51.0,5.6,-3.9,0.0,True,3.0,13.0,3.0,True
2018-03-21T00:00:00,0.0,0.0,2.8,-2.8,0.6,False,6.6,114.0,8.6,True
2018-04-02T00:00:00,9.1,127.0,12.8,-1.1,-1.1,True,14.0,152.0,15.2,True


Como estamos fazendo a junção pelo índice, uma maneira mais fácil é usar o método `join()` em vez de `merge()`. Observe que o parâmetro de sufixo agora é `lsuffix` para o sufixo do dataframe da esquerda e `rsuffix` para o sufixo do dataframe da direita:

In [29]:
valid_station.join(station_with_wesf, how='left', rsuffix='_?').query('WESF > 0').head()

,PRCP,SNOW,TMAX,TMIN,TOBS,inclement_weather,PRCP_?,SNOW_?,WESF,inclement_weather_?
date,,,,,,,,,,
2018-01-30T00:00:00,0.0,0.0,6.7,-1.7,-0.6,False,1.5,13.0,1.8,True
2018-03-08T00:00:00,48.8,NaN,1.1,-0.6,1.1,False,28.4,NaN,28.7,NaN
2018-03-13T00:00:00,4.1,51.0,5.6,-3.9,0.0,True,3.0,13.0,3.0,True
2018-03-21T00:00:00,0.0,0.0,2.8,-2.8,0.6,False,6.6,114.0,8.6,True
2018-04-02T00:00:00,9.1,127.0,12.8,-1.1,-1.1,True,14.0,152.0,15.2,True


As junções podem ser muito intensivas em recursos, então é uma boa ideia descobrir que tipo de junção você precisa usando operações de conjunto antes de tentar a própria junção. As operações de conjunto do `pandas` são realizadas no índice, então as colunas pelas quais faremos a junção precisam ser definidas como índice. Vamos voltar aos dataframes `weather` e `station_info` e definir as colunas de ID da estação como índice:

In [30]:
weather.set_index('station', inplace=True)
station_info.set_index('id', inplace=True)

A interseção nos dirá quais estações estão presentes em ambos os dataframes. O resultado será o índice ao realizar um inner join:

In [31]:
weather.index.intersection(station_info.index)

Index(['GHCND:US1CTFR0039', 'GHCND:US1NJBG0015', 'GHCND:US1NJBG0017',
       'GHCND:US1NJBG0018', 'GHCND:US1NJBG0023', 'GHCND:US1NJBG0030',
       'GHCND:US1NJBG0039', 'GHCND:US1NJBG0044', 'GHCND:US1NJES0018',
       'GHCND:US1NJES0024',
       ...
       'GHCND:US1NJBG0037', 'GHCND:USC00284987', 'GHCND:US1NJES0031',
       'GHCND:US1NJES0029', 'GHCND:US1NJMD0086', 'GHCND:US1NJMS0097',
       'GHCND:US1NJMN0081', 'GHCND:US1NJMD0088', 'GHCND:US1NJES0040',
       'GHCND:US1NYQN0029'],
      dtype='object', length=110)

A diferença de conjunto nos dirá o que perdemos de cada lado. Ao realizar um inner join, não perdemos nada do dataframe `weather`:

In [32]:
weather.index.difference(station_info.index)

Index([], dtype='object')

No entanto, perdemos 169 estações do dataframe `station_info`:

In [33]:
station_info.index.difference(weather.index)

Index(['GHCND:US1CTFR0022', 'GHCND:US1NJBG0001', 'GHCND:US1NJBG0002',
       'GHCND:US1NJBG0005', 'GHCND:US1NJBG0006', 'GHCND:US1NJBG0008',
       'GHCND:US1NJBG0011', 'GHCND:US1NJBG0012', 'GHCND:US1NJBG0013',
       'GHCND:US1NJBG0020',
       ...
       'GHCND:USC00308322', 'GHCND:USC00308749', 'GHCND:USC00308946',
       'GHCND:USC00309117', 'GHCND:USC00309270', 'GHCND:USC00309400',
       'GHCND:USC00309466', 'GHCND:USC00309576', 'GHCND:USW00014708',
       'GHCND:USW00014786'],
      dtype='object', length=169)

A diferença simétrica nos diz o que perdemos de ambos os lados. É a combinação das diferenças de conjunto em cada direção:

In [36]:
ny_in_name = station_info[station_info.index.str.contains('US1NY')]

ny_in_name.index.difference(weather.index).shape[0]\
+ weather.index.difference(ny_in_name.index).shape[0]\
== weather.index.symmetric_difference(ny_in_name.index).shape[0]

True

A união nos mostrará tudo o que estará presente após um full outer join. Observe que passamos os valores únicos do índice para garantir que possamos ver o número de estações que restarão:

In [42]:
weather.index.unique().union(station_info.index)

Index(['GHCND:US1CTFR0022', 'GHCND:US1CTFR0039', 'GHCND:US1NJBG0001',
       'GHCND:US1NJBG0002', 'GHCND:US1NJBG0003', 'GHCND:US1NJBG0005',
       'GHCND:US1NJBG0006', 'GHCND:US1NJBG0008', 'GHCND:US1NJBG0010',
       'GHCND:US1NJBG0011',
       ...
       'GHCND:USW00014708', 'GHCND:USW00014732', 'GHCND:USW00014734',
       'GHCND:USW00014786', 'GHCND:USW00054743', 'GHCND:USW00054787',
       'GHCND:USW00094728', 'GHCND:USW00094741', 'GHCND:USW00094745',
       'GHCND:USW00094789'],
      dtype='object', length=279)

Observe que a diferença simétrica é na verdade a união das diferenças de conjunto:

In [43]:
ny_in_name = station_info[station_info.index.str.contains('US1NY')]

ny_in_name.index.difference(weather.index).union(weather.index.difference(ny_in_name.index)).equals(
    weather.index.symmetric_difference(ny_in_name.index)
)

True

<hr>
<div>
    <a href="../ch_03/5-handling_data_issues.ipynb">
        <button>&#8592; Chapter 3</button>
    </a>
    <a href="./2-dataframe_operations.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<hr>